### AcadGild Assignment 12

1.​ Introduction
This assignment will help you to consolidate the concepts learnt in the session.

2.​ Problem Statement
Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
Rename the columns as per the description from this file:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Task:
Create a sql db from adult dataset and name it sqladb
1. Select 10 records from the adult sqladb
2. Show me the average hours per week of all men who are working in private sector
3. Show me the frequency table for education, occupation and relationship, separately
4. Are there any people who are married, working in private sector and having a masters
degree
5. What is the average, minimum and maximum age group for people working in
different sectors
6. Calculate age distribution by country
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
'capital-loss'

In [ ]:
# import the library SQLite
import sqlite3

# Load our regular libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#### Read the dataset and display the first few columns to see what we have to work with

In [ ]:
adult_data_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
#adult_data_df = pd.read_csv('adult.data')
display(adult_data_df.head(3))

#### Rename the columns as assigned
Notice that the final column <=50 etc... did not have a column name assigned in the reference above, so I just assigned it "income" because that seemed the logical thing to do :)


In [ ]:
adult_data_df.columns =["age","workclass","fnlwgt","education","education_num","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_week","native_country","income"]

#### Display the changes

In [ ]:
display(adult_data_df.head(3))

#### Start your Engine with sqlalchemy! :)

Notice that in the create_engine statement echo is set to 'True', this would NOT be done in a production environment, but is here so that the output can clearly be seen... what "echo=True" does is initiate Python's standard logging module so the output may be easily viewed.

In [ ]:
import sqlalchemy

from sqlalchemy import create_engine

engine = create_engine('sqlite:///sqladb', echo=True)

### Convert Pandas DataFrame to sql
Note: Be careful with the "if_exists='replace'" flag below in production databases. Obviously you would not want to overwrite something important! I simply used it here for convenience sake :)

In [ ]:
adult_data_df.to_sql('sqladb', engine, if_exists='replace')

#### Make a basic connection to the db

In [ ]:
connection = sqlite3.connect("sqladb")
cursor = connection.cursor()

#### Return basic column information of the DB


In [ ]:
cursor.execute('PRAGMA TABLE_INFO({})'.format("sqladb"))
cursor.fetchall()

#### Return total number of rows in DB

In [ ]:
cursor.execute('SELECT COUNT(*) FROM {}'.format("sqladb"))
cursor.fetchall()

##### Question 1: Select 10 records from the DB

In [ ]:
#cursor.execute('SELECT * FROM {tn} limit 10'. format(tn="sqladb"))
#output = cursor.fetchall()
#print(output)
#
# USE Pandas Print because it is cleaner
print(pd.read_sql_query('SELECT * FROM sqladb limit 10', connection))

In [ ]:
display(adult_data_df.head(3))

##### Question 2: Average hours per week of all men working in the private sector

In [ ]:
cursor.execute('SELECT avg(hours_per_week) FROM sqladb WHERE workclass=" Private"')
output = cursor.fetchall()
print(output)

##### Question 3a: Frequency table for education

In [ ]:
cursor.execute('SELECT education, COUNT(*) FROM sqladb GROUP BY education')
output = cursor.fetchall()
display(output)

##### Question 3b: Frequency table for occupation

In [ ]:
cursor.execute('SELECT occupation, COUNT(*) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 3c: Frequency table for relationship

In [ ]:
cursor.execute('SELECT relationship, COUNT(*) FROM sqladb GROUP BY relationship')
output = cursor.fetchall()
display(output)

##### Question 4: Are there any people who are married, working in private sector and having a masters degree

In [ ]:
#cursor.execute('SELECT * FROM sqladb WHERE education=" Masters" AND workclass=" Private" AND marital_status=" Married-civ-spouse"')
#output = cursor.fetchall()
#print(output)
#
# USE Pandas Print because it is cleaner
print(pd.read_sql_query('SELECT * FROM sqladb WHERE education=" Masters" AND workclass=" Private" AND marital_status=" Married-civ-spouse"', connection).head(10))

##### Just making a note for myself..

In [ ]:
cursor.execute('SELECT avg(age) AS average, 
               max(age) AS maximum, 
               min(age) AS minimum 
               FROM sqladb 
               GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 5a: The minimum age for working people by sector

In [ ]:
cursor.execute('SELECT occupation, min(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 5b: The maximum age for working people by sector

In [ ]:
cursor.execute('SELECT occupation, max(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 5c: The average age for working people by sector

In [ ]:
cursor.execute('SELECT occupation, avg(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 6a: Calculate minimum age distribution by country

In [ ]:
cursor.execute('SELECT native_country, min(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

##### Question 6b: Calculate maximum age distribution by country

In [ ]:
cursor.execute('SELECT native_country, max(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

##### Question 6c: Calculate average age distribution by country

In [ ]:
cursor.execute('SELECT native_country, avg(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

##### Question 7: Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [ ]:
cursor.execute('UPDATE sqladb SET net_capital_gain=capital_gain-capital_loss')

#### Use Pandas to print out a decent looking sql query from sqlite

In [ ]:
print(pd.read_sql_query("SELECT * FROM sqladb", connection).head(10))

In [ ]:
connection.close()